# completion_env

> Autocomplete helper: determine context of the token to be autocompleted

Adapted from the [stata_kernel version](https://github.com/kylebarron/stata_kernel/blob/master/stata_kernel/completions.py) (omitting mata-specific stuff).

In [ ]:
#| default_exp completion_env
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.utils import ending_delimiter, is_cr_delimiter
from fastcore.basics import patch_to
from enum import IntEnum
from typing import Tuple
import re

In [ ]:
#| export
class CompletionEnv():
    def __init__(self):
        """"""        
        # any non-space/"/= 'word' at the end of the string after the last ", =, or white space
        self.last_chunk = re.compile(
            r'[\s"=][^\s"=]*?\Z', flags=re.MULTILINE).search
        
        # Path completion
        self.path_search = re.compile(
            r'^(?P<fluff>.*")(?P<path>[^"]*)\Z').search

        # Magic completion
        self.magic_completion = re.compile(
            r'\A\*?%(?P<magic>\S*)\Z', flags=re.DOTALL + re.MULTILINE).match

        # Match context; this is used to determine if the line starts
        # with matrix or scalar. It also matches constructs like
        #
        #     (`=)?scalar(
        pre = (
            r'(cap(t|tu|tur|ture)?'
            r'|qui(e|et|etl|etly)?'
            r'|n(o|oi|ois|oisi|oisil|oisily)?)')
        kwargs = {'flags': re.MULTILINE}
        self.fcontext = {
            'function':
                re.compile(
                    r"(\s+|\=|`=)\s*(?P<name>\w+?)"
                    r"\([^\)]*?(?P<last_word>\w*)\Z", **kwargs).search,
        }
        self.context = {
            'line':
                re.compile(
                    r"^(?P<last_line>\s*({0}\s+)*(?P<first_word>\S+) .*?)\Z".format(pre),
                    **kwargs).search,
            'delimit_line':
                re.compile(
                    r"(?:\A|;)(?P<last_line>\s*({0}\s+)*(?P<first_word>[^\s;]+)\s[^;]*?)\Z".format(pre),
                    **kwargs).search
        }

In [ ]:
#| export
@patch_to(CompletionEnv)
def _scalar_f_pos_rcomp(self, code, r2chars):
    scalar_f = False
    funcontext = self.fcontext['function'](code)
    if funcontext:
        function = funcontext.group('name')
        if function == 'scalar':
            scalar_f = True
            pos = funcontext.start('last_word') if funcontext.start('last_word') else len(code)
            rcomp = "" if (r2chars[0:1] == ")" or r2chars == " )") else ")"
    if scalar_f:
        return True, pos, rcomp
    else:
        return False, None, None

In [ ]:
#| hide
test_instance = CompletionEnv()

In [ ]:
#| hide
code = """\
sysuse lifeexp
disp scalar("""
test_instance._scalar_f_pos_rcomp(code, "")

(True, 27, ')')

In [ ]:
#| hide
code = """\
sysuse lifeexp
disp scalar(te"""
test_instance._scalar_f_pos_rcomp(code, "")

(True, 27, ')')

In [ ]:
#| hide
code = """\
sysuse lifeexp
disp scalar( te"""
test_instance._scalar_f_pos_rcomp(code, "")

(True, 28, ')')

In [ ]:
#| hide
code = """\
sysuse lifeexp
disp scalar(
te"""
test_instance._scalar_f_pos_rcomp(code, "")

(True, 28, ')')

In [ ]:
#| hide
code = """\
sysuse lifeexp
disp
scalar(
te"""
test_instance._scalar_f_pos_rcomp(code, "")

(True, 28, ')')

In [ ]:
#| export
@patch_to(CompletionEnv)
def _start_of_last_chunk(self, code):
    search = self.last_chunk(code)
    return search.start() + 1 if search else 0

In [ ]:
#| hide
code = """\
sysuse lifeexp
disp
scalar(
te"""
test_instance._start_of_last_chunk(code)

28

In [ ]:
#| hide
from fastcore.test import test_eq

In [ ]:
#| hide
def _word(code):
    return code[test_instance._start_of_last_chunk(code):]

test_eq(_word("""use  00"""), """00""")

test_eq(_word("""use `00"""), """`00""")
test_eq(_word("""use $00"""), """$00""")
test_eq(_word("""use ${00"""), """${00""")
test_eq(_word("""use {00"""), """{00""")
test_eq(_word("""use /00"""), """/00""")

test_eq(_word("""use "00"""), """00""")
test_eq(_word("""use"00"""), """00""")
test_eq(_word("""use `"00"""), """00""")
test_eq(_word('''use `"00"'''), "")
test_eq(_word("""use `"00"'"""), "'")

test_eq(_word("""use """), "")
test_eq(_word("""use"""), "use")

test_eq(_word("""use `tes"""), """`tes""")
test_eq(_word("""use `tes'"""), """`tes'""")
test_eq(_word("""use ${tes}"""), """${tes}""")

test_eq(_word("\n".join(["use", "${tes}"])), "${tes}")

test_eq(_word("`=tes"), "tes")
test_eq(_word("disp `=tes"), "tes")

In [ ]:
#| export
@patch_to(CompletionEnv)
def _last_line_first_word(self, code, sc_delimit_mode=False):
    if sc_delimit_mode:
        linecontext = self.context['delimit_line'](code)
    else:
        linecontext = self.context['line'](code)
    if linecontext:
        last_line = linecontext.groupdict()['last_line']
        first_word = linecontext.groupdict()['first_word']
        return last_line, first_word
    else:
        return None, None

In [ ]:
#| hide
code = """\
sysuse lifeexp
list lex"""
test_instance._last_line_first_word(code, False)

('list lex', 'list')

In [ ]:
#| hide
code = """\
sysuse lifeexp
list lex"""
test_instance._last_line_first_word(code, True)

('sysuse lifeexp\nlist lex', 'sysuse')

In [ ]:
#| hide
code = """\
sysuse lifeexp;list lex"""
test_instance._last_line_first_word(code, True)

('list lex', 'list')

In [ ]:
#| hide
code = """\
sysuse lifeexp
lis"""
test_instance._last_line_first_word(code, False)

(None, None)

In [ ]:
#| hide
code = """\
#delimit;
sysuse lifeexp
list lex"""
test_instance._last_line_first_word(code, True)

('\nsysuse lifeexp\nlist lex', 'sysuse')

In [ ]:
#| export
class Env(IntEnum):
    MAGIC = -1     # magics, %x*
    GENERAL = 0    # varlist and/or file path
    LOCAL = 1      # `x* completed with `x*'
    GLOBAL = 2     # $x* completed with $x* or ${x* completed with ${x*}
    SCALAR = 4     # scalar .* x* completed with x* or scalar(x* completed with scalar(x*)
    MATRIX = 6     # matrix .* x* completed with x*
    SCALAR_VAR = 7 # scalars and varlist, scalar .* = x* completed with x*
    MATRIX_VAR = 8 # matrices and varlist, matrix .* = x* completed with x*
    MATA = 9       # inline or in mata environment

In [ ]:
#| export
@patch_to(CompletionEnv)
def get_env(self, 
            code: str, # Right-truncated to cursor position
            r2chars: str, # The two characters immediately after `code`, used to accurately determine rcomp
            starting_delimiter,
           ) -> Tuple[Env, int, str, str]:
    """Returns completions environment
    
    Returns
    -------
    env : Env    
    pos : int
        Where the completions start. This is set to the start of the word to be completed.
    out_chunk : str
        Word to match.
    rcomp : str
        How to finish the completion (defaulting to nothing):
        locals: '
        globals (if start with ${): }
        scalars: )
        scalars (if start with `): )'
    """
    lcode = code.lstrip()
    if self.magic_completion(lcode):
        pos = code.rfind("%") + 1
        env = Env.MAGIC
        rcomp = ""
        return env, pos, code[pos:], rcomp
    
    delimiter = ending_delimiter(code, starting_delimiter)
    env = Env.GENERAL
    rcomp = ''
    
    # Detect last "word" delimited by white space, a double-quote, or =.
    pos = self._start_of_last_chunk(code)

    if pos >= 1:
        # Figure out if current statement is a matrix or scalar
        # statement. If so, will add them to completions list.
        last_line, first_word = self._last_line_first_word(code, not is_cr_delimiter(delimiter))
        if first_word:
            equals_present = (last_line.find('=') > 0)
            if re.match(r'^sca(lar|la|l)?$', first_word): #.strip()
                env = Env.SCALAR_VAR if equals_present else Env.SCALAR
            elif re.match(r'^mat(rix|ri|r)?$', first_word): #.strip()
                env = Env.MATRIX_VAR if equals_present else Env.MATRIX

        # Constructs of the form scalar(x<tab> will be filled only
        # with scalars. This can be preceded by = or `=
        if env is Env.GENERAL:
            scalar_f, new_pos, new_rcomp = self._scalar_f_pos_rcomp(code, r2chars)
            if scalar_f:
                env = Env.SCALAR
                pos = new_pos
                rcomp = new_rcomp

    # Figure out if this is a local or global; env = 0 (default)
    # will suggest variables in memory.
    chunk = code[pos:]
    lfind = chunk.rfind('`')
    gfind = chunk.rfind('$')
    path_chars = any(x in chunk for x in ['/', '\\', '~'])

    if lfind >= 0 and (lfind > gfind):
        pos += lfind + 1
        env = Env.LOCAL
        rcomp = "" if r2chars[0:1] == "'" else "'"
    elif gfind >= 0 and not path_chars:
        bfind = chunk.rfind('{')
        if bfind >= 0 and (bfind == gfind+1):
            pos += bfind + 1
            env = Env.GLOBAL
            rcomp = "" if r2chars[0:1] == "}" else "}"
        else:
            env = Env.GLOBAL
            pos += gfind + 1

    closing_symbol = True #config.get('autocomplete_closing_symbol', 'False')
#     closing_symbol = closing_symbol.lower() == 'true'
    if not closing_symbol:
        rcomp = ''
    out_chunk = code[pos:]
    return env, pos, out_chunk, rcomp

In [ ]:
#| hide
test_instance.get_env("""\
scalar
list x""", "", ";")

(<Env.SCALAR: 4>, 12, 'x', '')

In [ ]:
#| hide
test_eq(
    [test_instance.get_env("""\
#delimit;
scalar
list x""", "", None)[i] for i in [0, 2, 3]],
    [test_instance.get_env("""\
scalar
list x""", "", ";")[i] for i in [0, 2, 3]])

In [ ]:
test_eq(
    test_instance.get_env("`", "", None)[0:3],
    (Env.LOCAL, 1, ""))

In [ ]:
#| hide
test_eq(
    test_instance.get_env("disp 1\n`", "", None)[0],
    Env.LOCAL)

In [ ]:
test_eq(
    test_instance.get_env("*%e", "", None)[0:2],
    (Env.MAGIC, 2))

In [ ]:
#| hide
test_eq(
    test_instance.get_env("%e", "", None)[0:2],
    (Env.MAGIC, 1))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()